# Limiting Course-GL mapping to only the specified fiscal years in course attributes

## What fiscal years are specified in the course attributes?

In [3]:
use FinancialReporting;
select distinct fiscal_year from STAGING.ATTRIBUTE_COURSE where hierarchy_version_id = 133 and attribute_version_id = 150;
select distinct Year YR_CDE, UPPER(LEFT(Fiscal_Term, 2)) TRM_CDE from INPUT.PERIODS_BY_DAY
where Fiscal_Year in (
    select fiscal_year from STAGING.ATTRIBUTE_COURSE where hierarchy_version_id = 133 and attribute_version_id = 150
    );

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.090

fiscal_year
FY 2018-2019


YR_CDE,TRM_CDE
2018,FA
2018,SU
2019,SP


## Generate mapping: current state

In [1]:
use FinancialReporting;
exec STAGING.GENERATE_COURSE_GL_MAPPING 133, 150;

(25879 rows affected)

(4604 rows affected)

(25879 rows affected)

Total execution time: 00:00:28.728

## Save current state

In [1]:
use TmsEPrd;
select * into ##jjj_current from TWU_VENA_COURSE_GL_MAPPING where HIERARCHY_VERSION_ID = 133 and ATTRIBUTE_VERSION_ID = 150;

(25879 rows affected)

Total execution time: 00:00:00.204

## ALTER stored procedure STAGING.GENERATE_COURSE_GL_MAPPING with future logic

In [8]:
-- Use FinancialReporting;
-- ALTER PROCEDURE STAGING.GENERATE_COURSE_GL_MAPPING

Commands completed successfully.

Total execution time: 00:00:00.022

## Generate mapping: future state

In [3]:
use FinancialReporting;
exec STAGING.GENERATE_COURSE_GL_MAPPING 133, 150;

(1 row affected)

(3 rows affected)

(4604 rows affected)

(25879 rows affected)

(4604 rows affected)

Total execution time: 00:00:07.434

## Save future state

In [2]:
-- Save future state.
use TmsEPrd;
select * into ##jjj_future from TWU_VENA_COURSE_GL_MAPPING where HIERARCHY_VERSION_ID = 133 and ATTRIBUTE_VERSION_ID = 150;

(4604 rows affected)

Total execution time: 00:00:00.086

## What does current vs future look like for the specified fiscal year FY 2018-2019?

In [15]:
use TmsEPrd;

declare
    @hierarchyId INT = 133,
    @attributeId INT = 150;

declare @yearterms table (YR_CDE CHAR(4), TRM_CDE CHAR(2));
insert into @yearterms values
('2018', 'SU'),
('2018', 'FA'),
('2019', 'SP');

select * from @yearterms;

select '1 Num rows in current' [Key], count(*) [Value]
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)

union

select '2 Num rows in future' [Key], count(*) [Value]
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)

union

select '3 Num rows in current but not in future' [Key], count(*) [Value] from (
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
except
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
) x

union

select '4 Num rows in future but not in current' [Key], count(*) [Value] from (
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
except
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
) x

(3 rows affected)

(3 rows affected)

(4 rows affected)

Total execution time: 00:00:00.560

YR_CDE,TRM_CDE
2018,SU
2018,FA
2019,SP


Key,Value
1 Num rows in current,4604
2 Num rows in future,4604
3 Num rows in current but not in future,0
4 Num rows in future but not in current,0


## What does it look like for calendar year 2018?

In [16]:
use TmsEPrd;

declare
    @hierarchyId INT = 133,
    @attributeId INT = 150;

declare @yearterms table (YR_CDE CHAR(4), TRM_CDE CHAR(2));
insert into @yearterms values
('2018', 'SP'),
('2018', 'SU'),
('2018', 'FA');

select * from @yearterms;

select '1 Num rows in current' [Key], count(*) [Value]
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)

union

select '2 Num rows in future' [Key], count(*) [Value]
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)

union

select '3 Num rows in current but not in future' [Key], count(*) [Value] from (
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
except
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
) x

union

select '4 Num rows in future but not in current' [Key], count(*) [Value] from (
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
except
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
) x

(3 rows affected)

(3 rows affected)

(4 rows affected)

Total execution time: 00:00:00.433

YR_CDE,TRM_CDE
2018,SP
2018,SU
2018,FA


Key,Value
1 Num rows in current,4567
2 Num rows in future,2703
3 Num rows in current but not in future,1864
4 Num rows in future but not in current,0


## What does it look like for calendar year 2019?

In [17]:
use TmsEPrd;

declare
    @hierarchyId INT = 133,
    @attributeId INT = 150;

declare @yearterms table (YR_CDE CHAR(4), TRM_CDE CHAR(2));
insert into @yearterms values
('2019', 'SP'),
('2019', 'SU'),
('2019', 'FA');

select * from @yearterms;

select '1 Num rows in current' [Key], count(*) [Value]
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)

union

select '2 Num rows in future' [Key], count(*) [Value]
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)

union

select '3 Num rows in current but not in future' [Key], count(*) [Value] from (
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
except
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
) x

union

select '4 Num rows in future but not in current' [Key], count(*) [Value] from (
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_future
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
except
select MAPPING_TYPE, YR_CDE, TRM_CDE, CRS_CDE, CRS_CDE_HIER, LOC_CDE, ACCT_CDE
from ##jjj_current
where HIERARCHY_VERSION_ID = @hierarchyId and ATTRIBUTE_VERSION_ID = @attributeId
and concat(YR_CDE, TRM_CDE) in (select concat(YR_CDE, TRM_CDE) from @yearterms)
) x

(3 rows affected)

(3 rows affected)

(4 rows affected)

Total execution time: 00:00:00.705

YR_CDE,TRM_CDE
2019,SP
2019,SU
2019,FA


Key,Value
1 Num rows in current,4513
2 Num rows in future,1901
3 Num rows in current but not in future,2612
4 Num rows in future but not in current,0
